# 필요 라이브러리 다운및 import

In [1]:
# 로컬에 깔려서 버전바뀌니까 그만 까세요!

# !pip install mxnet
# !pip install gluonnlp pandas tqdm
# !pip install sentencepiece
# !pip install transformers==3.0.2
# !pip install torch

In [2]:
# !pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

In [3]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
device = torch.device("cuda:0")
print(torch.cuda.is_available())

True


In [4]:
print(torch.cuda.current_device())
print(torch.cuda.device_count())

0
10


In [5]:
from kobert import get_tokenizer
from kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [6]:
bertmodel, vocab = get_pytorch_kobert_model()

using cached model. /home/jupyter-j7d203/Untitled Folder/.cache/kobert_v1.zip
using cached model. /home/jupyter-j7d203/Untitled Folder/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [7]:
# from google.colab import drive
# drive.mount('/content/gdrive'
# )

In [8]:
# cd/content/gdrive/My Drive/Project

# 데이터 전처리

In [9]:
import pandas as pd
df = pd.read_csv('./text_data5.csv',encoding='cp949')

In [10]:
len(df)

35510

In [11]:
df['emotion'].unique()

array([4, 1, 0, 2, 3])

In [12]:
df['emotion'].value_counts()

4    10953
0     8254
3     5824
1     5576
2     4903
Name: emotion, dtype: int64

In [13]:
index = df[df['emotion']==5].index

In [14]:
print(index)

Int64Index([], dtype='int64')


In [15]:
df = df.drop(index)

In [16]:
df['emotion'].value_counts()

4    10953
0     8254
3     5824
1     5576
2     4903
Name: emotion, dtype: int64

In [17]:
df.sample(n=10)

,dialog,emotion
10298,네? 그런 게 어딨어요?,1
27321,이번 프로젝트에서 멋진 선배를 만나게 되었어. 믿고 배울만한 사수를 얻은 것 같아.,3
2341,국내 최고 의사에게 치료받을 수 있게 되었어.,3
16553,시험 봤는데 답을 밀려 썼나 봐. 말도 안 되는 점수가 나왔어.,4
4442,나 오늘도 남자친구랑 싸우고 왔어. 벌써 몇 번째 싸우는 건지 모르겠어.,4
28038,이제야 노후 연금을 알아보고 준비하고 있어. 너무 늦은 건 아닐지 걱정돼.,2
8232,내 친구의 남자 친구가 다른 사람과 다정하게 걸어가는 모습을 봤어.,4
32417,친구들로부터 그녀도 나에게 관심 있어 보인다는 말을 들었어.,3
20363,"어서, 멱살 풀고 자식아!",0
16170,속편한 소리하고 있으시네,0


In [18]:
import random
import re

# 한글만 남기고 나머지는 삭제
def get_only_hangul(line):
	parseText= re.compile('/ ^[ㄱ-ㅎㅏ-ㅣ가-힣]*$/').sub('',line)

	return parseText



########################################################################
# Synonym replacement
# Replace n words in the sentence with synonyms from wordnet
########################################################################
def synonym_replacement(words, n):
	new_words = words.copy()
	random_word_list = list(set([word for word in words]))
	random.shuffle(random_word_list)
	num_replaced = 0
	for random_word in random_word_list:
		synonyms = get_synonyms(random_word)
		if len(synonyms) >= 1:
			synonym = random.choice(list(synonyms))
			new_words = [synonym if word == random_word else word for word in new_words]
			num_replaced += 1
		if num_replaced >= n:
			break

	if len(new_words) != 0:
		sentence = ' '.join(new_words)
		new_words = sentence.split(" ")

	else:
		new_words = ""

	return new_words


def get_synonyms(word):
	synomyms = []

	try:
		for syn in wordnet[word]:
			for s in syn:
				synomyms.append(s)
	except:
		pass

	return synomyms

########################################################################
# Random deletion
# Randomly delete words from the sentence with probability p
########################################################################
def random_deletion(words, p):
	if len(words) == 1:
		return words

	new_words = []
	for word in words:
		r = random.uniform(0, 1)
		if r > p:
			new_words.append(word)

	if len(new_words) == 0:
		rand_int = random.randint(0, len(words)-1)
		return [words[rand_int]]

	return new_words

########################################################################
# Random swap
# Randomly swap two words in the sentence n times
########################################################################
def random_swap(words, n):
	new_words = words.copy()
	for _ in range(n):
		new_words = swap_word(new_words)

	return new_words

def swap_word(new_words):
	random_idx_1 = random.randint(0, len(new_words)-1)
	random_idx_2 = random_idx_1
	counter = 0

	while random_idx_2 == random_idx_1:
		random_idx_2 = random.randint(0, len(new_words)-1)
		counter += 1
		if counter > 3:
			return new_words

	new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1]
	return new_words

########################################################################
# Random insertion
# Randomly insert n words into the sentence
########################################################################
def random_insertion(words, n):
	new_words = words.copy()
	for _ in range(n):
		add_word(new_words)
	
	return new_words


def add_word(new_words):
	synonyms = []
	counter = 0
	while len(synonyms) < 1:
		if len(new_words) >= 1:
			random_word = new_words[random.randint(0, len(new_words)-1)]
			synonyms = get_synonyms(random_word)
			counter += 1
		else:
			random_word = ""

		if counter >= 10:
			return
		
	random_synonym = synonyms[0]
	random_idx = random.randint(0, len(new_words)-1)
	new_words.insert(random_idx, random_synonym)



def EDA(sentence, alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.1, p_rd=0.1, num_aug=9):
	sentence = get_only_hangul(sentence)
	words = sentence.split(' ')
	words = [word for word in words if word is not ""]
	num_words = len(words)

	augmented_sentences = []
	num_new_per_technique = int(num_aug/4) + 1

	n_sr = max(1, int(alpha_sr*num_words))
	n_ri = max(1, int(alpha_ri*num_words))
	n_rs = max(1, int(alpha_rs*num_words))

	# sr
	for _ in range(num_new_per_technique):
		a_words = synonym_replacement(words, n_sr)
		augmented_sentences.append(' '.join(a_words))

	# ri
	for _ in range(num_new_per_technique):
		a_words = random_insertion(words, n_ri)
		augmented_sentences.append(' '.join(a_words))

	# rs
	for _ in range(num_new_per_technique):
		a_words = random_swap(words, n_rs)
		augmented_sentences.append(" ".join(a_words))

	# rd
	for _ in range(num_new_per_technique):
		a_words = random_deletion(words, p_rd)
		augmented_sentences.append(" ".join(a_words))

	augmented_sentences = [get_only_hangul(sentence) for sentence in augmented_sentences]
	random.shuffle(augmented_sentences)

	if num_aug >= 1:
		augmented_sentences = augmented_sentences[:num_aug]
	else:
		keep_prob = num_aug / len(augmented_sentences)
		augmented_sentences = [s for s in augmented_sentences if random.uniform(0, 1) < keep_prob]

	augmented_sentences.append(sentence)

	return augmented_sentences

<>:132: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:132: SyntaxWarning: "is not" with a literal. Did you mean "!="?
/tmp/ipykernel_1656090/4290562359.py:132: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  words = [word for word in words if word is not ""]


In [19]:
data_list = []
for q,label in zip(df['dialog'], df['emotion']) :
  data = []
  data.append(q)
  data.append(str(label))
  data_list.append(data)
  for i in EDA(q):
    data_two = []
    data_two.append(i)
    data_two.append(str(label))
    data_list.append(data_two)
print(len(data_list))

  

390610


In [20]:
print(data_list[0])
print(data_list[1])
print(data_list[1000])

[' ... 그래. 알았어. 그만 하자. 내가 잘못했어. 그만해.', '4']
['... 그래. 알았어. 그만 하자. 내가 잘못했어. 그만해.', '4']
['..왜? ', '1']


In [21]:
from sklearn.model_selection import train_test_split

dataset_train, dataset_test = train_test_split(data_list, test_size = 0.25, random_state =0)

In [22]:
print(len(dataset_train))
print(len(dataset_test))

292957
97653


# BERT DATASET

In [23]:
class BERTDataset(Dataset):
  def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len, pad, pair):
    transform = nlp.data.BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

    self.sentences = [transform([i[sent_idx]]) for i in dataset]
    self.labels = [np.int32(i[label_idx])for i in dataset]

  def __getitem__(self, i):
    return (self.sentences[i] + (self.labels[i],))

  def __len__(self):
    return(len(self.labels))

parameter

In [24]:
max_len = 64
batch_size = 128
warmup_ratio = 0.1
num_epochs = 3
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-5


In [25]:
!nvidia-smi

Wed Sep 28 19:46:48 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  On   | 00000000:03:00.0 Off |                    0 |
| N/A   32C    P0    34W / 250W |  13025MiB / 32768MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-PCIE...  On   | 00000000:04:00.0 Off |                    0 |
| N/A   

토큰화 과정

In [26]:
tokenizer = get_tokenizer()

using cached model. /home/jupyter-j7d203/Untitled Folder/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [27]:
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

In [28]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [29]:
data_train[2]

(array([   2, 1397, 1804, 2640, 6607, 7925, 7990, 5561, 1820, 2075, 7088,
        3592, 7782, 5757, 5770, 1185, 3739, 6896, 2934,  736,  921, 1422,
        3647, 5936, 4206, 5439, 2709,  517,   54, 1185, 5400, 1917, 5859,
        4832, 7788,  517,   54,    3,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(38, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 3)

In [30]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

# BERT MODEL

In [31]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=5,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids))
        if self.dr_rate:
            out = self.dropout(pooler)
        else:
            out = pooler
        return self.classifier(out)

In [32]:
model = BERTClassifier(bertmodel,  dr_rate=0.5)

In [33]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [34]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [35]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [36]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [37]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

# Training & save model

In [38]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long()
        segment_ids = segment_ids.long()
        valid_length= valid_length
        label = label.long()
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):
        token_ids = token_ids.long()
        segment_ids = segment_ids.long()
        label = label.long()
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

  0%|          | 1/2289 [00:08<5:10:41,  8.15s/it]

epoch 1 batch id 1 loss 1.6576343774795532 train acc 0.234375


  9%|▉         | 201/2289 [11:51<2:01:06,  3.48s/it]

epoch 1 batch id 201 loss 1.2144496440887451 train acc 0.4005752487562189


 18%|█▊        | 401/2289 [23:20<1:46:10,  3.37s/it]

...

epoch 3 batch id 2201 loss 0.1034935787320137 train acc 0.9805166685597456


100%|██████████| 2289/2289 [2:59:28<00:00,  4.70s/it]

epoch 3 train acc 0.9806193467941582



100%|██████████| 763/763 [23:43<00:00,  1.87s/it] 

epoch 3 test acc 0.9802969154316632


In [39]:
torch.save(model.state_dict(), './model6.h5')

# Load Model & Predict

In [41]:
model = BERTClassifier(bertmodel,  dr_rate=0.5)
model.load_state_dict(torch.load('./model6.h5'))
model.eval()

BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True

In [82]:
def predict(predict_sentence):
  
  data = [predict_sentence, '0']
  dataset_another = [data]

  another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
  test_dataloader = torch.utils.data.DataLoader(another_test, batch_size = batch_size, num_workers=5)

  model.eval()

  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
    token_ids = token_ids.long()
    segment_ids = segment_ids.long()

    valid_length = valid_length
    label = label.long()

    out = model(token_ids, valid_length, segment_ids)

    test_eval=[]

    for i in out:
      logits = i
      logits = logits.detach().numpy()
    
      if np.argmax(logits) == 0:
        test_eval.append("분노")
      elif np.argmax(logits) == 1:
        test_eval.append("놀람")
      elif np.argmax(logits) == 2:
        test_eval.append("불안")
      elif np.argmax(logits) == 3:
        test_eval.append("행복")
      elif np.argmax(logits) == 4:
        test_eval.append("슬픔")
      # elif np.argmax(logits) == 5:
      #   test_eval.append("당황이")
   
#     print(">> 입력하신 내용에서 "+ test_eval[0] + " 느껴집니다.")
#     print(min(logits))
#     print(logits)
    min_logits = min(logits)
    if min_logits < 0:
        for q in range(len(logits)):
            logits[q]+=abs(min_logits)
    total = sum(logits)
    emotions_scores =[]
    for w in range(len(logits)):
        emotions_scores.append(round(((logits[w]/total) * 100), 2))
#     print(emotions_scores)
#     print(sum(emotions_scores))
    return emotions_scores, test_eval[0]

In [83]:
predict("오늘 친구들과 함께 밥을 먹었다. 내가 좋아하는 김치찌개라서 기대했는데, 엄마가 해준 김치찌개가 더 맛있어서 기대 이하였다. ")

([16.36, 6.15, 0.0, 19.26, 58.24], '슬픔')